In [1]:
import scipy.io as sio
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt


In [2]:
class SignalFeatures():
    # signal basic parameters
    rest_original_signal = []   # original signal in 1 row []
    active_original_signal = [] # original signal in 1 row []
    fs = 0

    rest_signal_len = 0     # original signal length
    active_signal_len = 0   # original signal length
    # data segment list, 2D list
    # initiate in signal_segment function
    rest_signal_segment = []
    active_signal_segment = []
    # init function
    def __init__(self, rest_signal, active_signal, sample_freq):
        # input signal: 1D list []
        self.rest_original_signal = rest_signal
        self.active_original_signal = active_signal
        fs = sample_freq

        self.rest_signal_len = len(rest_signal)
        self.active_signal_len = len(active_signal)
        pass
    # signal split function
    def signal_segment(self, window_len, step_len):
        # split rest signal
        if self.rest_signal_len < window_len:
            print("Rest signal length is below the window length")
            self.rest_signal_segment.append(self.rest_original_signal)
        else:
            for i in range((self.rest_signal_len - window_len)//step_len + 1):
                self.rest_signal_segment.append(self.rest_original_signal[i*step_len : i*step_len + window_len])
                pass
            pass

        # split active signal
        if self.active_signal_len < window_len:
            print("active signal length is below the window length")
            self.active_signal_segment.append(self.active_original_signal)
        else:
            for i in range((self.active_signal_len - window_len)//step_len + 1):
                self.active_signal_segment.append(self.active_original_signal[i*step_len : i*step_len + window_len])
                pass
            pass

    pass


class FMGFeatures(SignalFeatures):

    pass


class sEMGFeatures(SignalFeatures):

    pass

In [3]:
# 读取预实验路径下的几次实验数据
# file path format: 'D:\\folder path\\subject name\\label_(rest/active).mat'
file_folder_path = 'D:\code\data\iFEMG_data_set'
subjects = ['\zpk1', '\zpk2', '\zpk3', '\zpk4', '\zpk5', '\zpk6']
mat_name = ['d0', 'd2', 'd5', 'd7', 'dm']

signal_df = pd.DataFrame(columns = ('subject_name', 'rest_signal', 'active_signal', 'label', 'sensor_channel'))

for i in subjects:
    for j in mat_name:
        # print('processing: ', i + j)
        # build data frame, every row has a unique label
        signal_df = signal_df.append({'subject_name' : i,
                                    'rest_signal': sio.loadmat(file_folder_path + i + '\\' + j + '_rest.mat')[j + '_rest'],
                                    'active_signal': sio.loadmat(file_folder_path + i + '\\' + j + '_active.mat')[j + '_active'], 
                                    'label' : j,
                                    'sensor_channel' : 'bicps_br'}, ignore_index=True)

    pass

In [11]:
# 使用boolean值索引出某一名受试者的实验数据
data = signal_df.loc[signal_df.loc[:, 'subject_name'] == '\zpk1']

#rest_FMG = data.loc[0, 'rest_signal'][:, 0]
#rest_sEMG = data.loc[0, 'rest_signal'][:, 1]
#active_FMG = data.loc[0, 'active_signal'][:, 0]
sEMG1 = SignalFeatures(data.loc[0, 'rest_signal'][:, 1], data.loc[0, 'active_signal'][:, 1], 1223)
print(len(sEMG1.active_signal_segment))
sEMG1.signal_segment(1223, 500)
print(len(sEMG1.active_signal_segment))



332
415


In [5]:
for row in data.itertuples():
    print(type(row))

<class 'pandas.core.frame.Pandas'>
<class 'pandas.core.frame.Pandas'>
<class 'pandas.core.frame.Pandas'>
<class 'pandas.core.frame.Pandas'>
<class 'pandas.core.frame.Pandas'>


In [1]:
class Person:
    def __new__(cls: type[Self]) -> Self:
        print("this is new")
        pass
    def __init__(self) -> None:
        print("this is init")
        pass
    def __del():
        pass
    pass

IndentationError: expected an indented block (Temp/ipykernel_3336/3571261461.py, line 9)